## The Cholera dataset
In this notebook we'll explore cholera outbreak data (2010-2019) for sub-Saharan Africa available [here](https://github.com/HopkinsIDD/cholera_outbreaks_ssa/blob/main/reference_data/outbreak_data.csv). Further metadata about this dataset can be found in the repo's [README.md](https://github.com/HopkinsIDD/cholera_outbreaks_ssa) file. This dataset is sourced from [Zheng et al. (2022)](https://www.sciencedirect.com/science/article/pii/S1201971222003034), but for the purposes of this work, we'll use this dataset purely as a source of outbreak data. 

In [ ]:
import geopandas as gpd
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

hv.extension("bokeh")

%output widgets='live' holomap='scrubber'

In [ ]:
cholera_data = pd.read_csv("data/outbreak_data.csv")
cholera_data

## A quick examination of the dataset

Looking at a summary of the dataset, columns, datatypes and missing values. 

In [ ]:
cholera_data.columns

In [ ]:
cholera_data.describe()

In [ ]:
cholera_data.dtypes

In [ ]:
cholera_data.isnull().values.any()

In [ ]:
cholera_data.isnull().sum()

Above we can see that the majority of all columns do not have missing data, with the exception of `total_deaths`, `cfr` (the case fatality rate of an outbreak) and `total_confirmed_cases.` This makes sense as not all outbreaks will have confirmed deaths, and some suspected outbreaks may not have any confirmed cases. Below, we'll look at these missing values a bit closer. 

In [ ]:
cholera_data[cholera_data.isnull().any(1)]

## Data cleaning and preparation
Some pre-processing of the cholera data, to make it easier to work with other datasets. 

In [ ]:
cholera_data = cholera_data.rename(columns={"country": "ISO3"})

In [ ]:
# to group by year and month, we'll focus on the extract the year value from the 'start_date'
cholera_data["s_Date"] = pd.to_datetime(cholera_data["start_date"], format="%m/%d/%Y")
cholera_data["s_year"] = cholera_data["s_Date"].dt.year
cholera_data

In [ ]:
cholera_data.columns

## Exploratory data analysis 

In [ ]:
overall_trends = (
    cholera_data.groupby("s_year")
    .sum()
    .loc[:, ["total_suspected_cases", "total_confirmed_cases", "total_deaths"]]
)
overall_trends

In [ ]:
overall_trends.hvplot.line(
    x="s_year",
    y=["total_suspected_cases", "total_confirmed_cases", "total_deaths"],
    rot=90,
)

Interestingly, if we zoom into the chart we find that `total_deaths` are sometimes greater than `total_confirmed_cases.` We should keep in mind, that in this line chart everything is aggregated by year (and not by country) so that this may be due to differences in quality of surveillance records across different areas. Regardless, it's something to keep in mind. 

In either case, we observe 2 distinct peaks of `total_suspected_cases`, one in 2012 and the other in 2016-2017 - the latter supported by an increase in `total_confirmed_cases.`

Grouping annual `total_suspected_cases` by country. 

In [ ]:
yearly_cases = (
    cholera_data.groupby(["s_year", "ISO3"])
    .sum()["total_suspected_cases"]
    .reset_index()
)
yearly_cases

In [ ]:
# using hvplot to create a holoviews plot, but could also use holoviews itself

bar_chart = yearly_cases.hvplot.bar(
    x="ISO3",
    y="total_suspected_cases",
    by="s_year",
    cmap="Category20",
    stacked=True,
    legend="right",
    width=800,
    rot=90,
)

bar_chart

In the stacked bar chart above we above observe that some countries see repeated outbreaks more often than others. Understanding this geographic distribution and if these countries are located nearer to each other will be helpful in understanding outbreak dynamics.

Which countries are most affected? 

In [ ]:
country_agg = (
    cholera_data.groupby("ISO3").sum().loc[:, ["total_suspected_cases", "total_deaths"]]
)
country_agg = country_agg.sort_values("total_suspected_cases", ascending=False).head(10)

country_agg.loc[:, ["total_deaths", "total_suspected_cases"]].iloc[::-1].hvplot.barh(
    colormap="coolwarm_r", stacked=False, legend="bottom_right", height=600
)

## Giving geographic context

Now we'll want to merge the yearly cases against the `admin0` boundaries so that we can map the distribution over time. Because both datasets share the 3-digit `ISO3` country code, we can merge them together on that column. 

We'll need to add administrative boundaries to provide geospatial context. We'll use the ICPAC Administrative Level 0 boundaries available [here](https://geoportal.icpac.net/layers/geonode:afr_g2014_2013_0/metadata_detail) and read them in with `geopandas`. This may take a few seconds.

In [ ]:
admin0_gdf = gpd.read_file(
    "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_0&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
)
admin0_gdf

Checking the coordinate reference system of the `admin0` administrative boundary dataset. 

In [ ]:
admin0_gdf.crs

In [ ]:
merged_df = pd.merge(admin0_gdf, yearly_cases, how="inner", on="ISO3")
merged_df

As we did an `inner` merge, this will keep only those countries within the `admin0` dataset that _also_ have records in the cholera outbreak dataset. 

## Mapping Cholera outbreaks from 2010-2019 at the national level

Below, we explore two different approaches (`hvplot` vs. `plotly`) to mapping cholera outbreaks over time. This is two get a better sense, geographically, of where repeated outbreaks are occuring and to visualize any spatial autocorrelation between them (i.e., are nations repeatedly experiencing outbreaks in closer proximity to each other?) 

When displaying the `hvplot` below, a new window will open, but the map will be interactive. 

In [ ]:
yearly_total_cases = merged_df.hvplot(
    c="total_suspected_cases",
    width=900,
    height=500,
    geo=True,
    groupby="s_year",
    hover_cols=["total_suspected_cases", "ISO3"],
    cmap="viridis",
    hover_fill_color="white",
    line_width=1,
    alpha=0.5,
    title="Yearly Total Suspected Cases",
    tiles="CartoLight",
    widget_location="bottom",
)

# yearly_total_cases ##still won't show up inline
yearly_total_cases.show()  ## will open a new window

Here, we visualize the same dataset with `plotly` - some refinement could be done on the visualization - but we see a central focal point around the Democratic Republic of the Congo (ISO3:`COD`). This is supported by the earlier bar chart we developed earlier highlighting that COD was the country that experienced the greatest amount of `total_suspected_cases.`

In [ ]:
yearly_snapshot = px.choropleth(
    yearly_cases,
    locations="ISO3",
    color="total_suspected_cases",
    hover_name="ISO3",
    color_continuous_scale=px.colors.sequential.Plasma,
    animation_frame="s_year",
    animation_group="ISO3",
    range_color=[0, 100000],
)

yearly_snapshot.show()

TODO: 
* look into warnings 
* explore distortion issue with `hvplot` mapping during some time intervals
* explore other values within the dataset - is `total_suspected_cases` the most appropriate? Would like to explore population density values, size, and duration of outbreaks recorded